In [1]:
import findspark
findspark.init("/home/chenekv/spark-3.5.4-bin-hadoop3-scala2.13")

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, countDistinct

#Initialize and read file
spark = SparkSession.builder \
    .appName("UserBalanceAnalysis") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

file_path = "/home/chenekv/code/jupyterlab/Purchase Redemption Data/user_balance_table.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

24/12/21 17:14:21 WARN Utils: Your hostname, chenekv-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
24/12/21 17:14:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/21 17:14:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

In [2]:
spark.sparkContext.setLogLevel("ERROR")

In [3]:
daily_totals = df.groupBy("report_date").agg(
    sum("total_purchase_amt").alias("total_inflow"),
    sum("total_redeem_amt").alias("total_outflow")
)

#Result
daily_totals.show()

output_path_task1 = "/home/chenekv/code/jupyterlab/daily_totals_output"
daily_totals.write.csv(output_path_task1, header=True)

+-----------+------------+-------------+
|report_date|total_inflow|total_outflow|
+-----------+------------+-------------+
|   20140717|   253011280|    298279385|
|   20140627|   264282703|    399444352|
|   20140515|   313367089|    238307643|
|   20140314|   315897431|    311575572|
|   20140114|   356907128|    159778389|
|   20140714|   254797524|    284753279|
|   20140501|   193045106|    143362755|
|   20140519|   259077930|    293791406|
|   20131014|   130315300|     40652625|
|   20140228|   428721754|    322030204|
|   20140113|   447207050|    178923772|
|   20130921|    32406340|     26596179|
|   20130831|    47655303|     22012016|
|   20130807|    43908081|     29708706|
|   20131110|   164813471|     85293644|
|   20130912|    68573684|     45147220|
|   20130703|    27270770|      5953867|
|   20140403|   363877120|    266605457|
|   20131210|   179471612|    147147341|
|   20130822|    89130737|     25000672|
+-----------+------------+-------------+
only showing top

In [4]:
august_data = df.filter((col("report_date") >= 20140801) & (col("report_date") <= 20140831))

user_active_days = august_data.groupBy("user_id").agg(
    countDistinct("report_date").alias("active_days")
)

active_users = user_active_days.filter(col("active_days") >= 5)

active_user_count = active_users.count()

print(f"2014年8月活跃用户总数: {active_user_count}")

[Stage 8:=======>                                                   (1 + 7) / 8]

2014年8月活跃用户总数: 12767
